In [46]:
# !pip install -r requirements.txt

In [47]:
import numpy as np
import random
import trimesh as tri
import igl
from itertools import repeat

data_dir = "data/"
files = [
    #"beetle.obj",
    "ChineseLion.obj",
    #"Femur.obj",
    #"Foot.obj",
    #"happy.obj",
    #"Hat.obj",
    "HumanBrain.obj",
    #"HumanFace.obj",
    #"Nefertiti.obj",
    "OldMan.obj",
    #"rocker-arm.obj",
    "spot_triangulated.obj"
    #"StanfordBunny.obj",
    #"xyzrgb_dragon.obj"
]

In [48]:
def display_cut(vertex_positions, indicies, full_cut):
    trimesh = tri.Trimesh(vertices=vertex_positions, faces=indicies) 
    scene = tri.Scene(trimesh)

    for cut in full_cut:
        cut_vertices = vertex_positions[cut]
        range = np.arange(0, len(cut))
        line_range= tri.path.entities.Line(points=range,color=np.array([[255,0,0,255]]))
        path1 = tri.path.path.Path3D(entities=[line_range], vertices=cut_vertices)
        scene.add_geometry(path1)

    M = np.sum(np.mean(vertex_positions[indicies], axis=1), axis=1)
    colors = tri.visual.interpolate(M, color_map='viridis')
    trimesh.visual.face_colors = colors

    scene.show(flags = {'cull': False}, viewer='gl' )

    return colors

def remove_triangles_adjacent_to_path(indicies, cut):
    for sub_cut in cut:
        if (sub_cut[0] != sub_cut[-1]): 
            rmi = set()
            for i in range(len(sub_cut)):
                v = sub_cut[i]
                adjacent_triangle_indicies = np.where(np.sum(indicies==v, axis=1)>0)[0]
                rmi.update(adjacent_triangle_indicies)
            indicies = np.delete(indicies, list(rmi), axis=0)
    return indicies

def display_parameterized_geometry(vertex_positions, indicies, colors) :
    trimesh = tri.Trimesh(vertices=vertex_positions, faces=indicies) 
    trimesh.visual.face_colors = colors
    scene = tri.Scene(trimesh)
    scene.show(flags = {'wireframe':True, 'cull': False}, line_settings={'line_width': 0.5}, viewer='gl' )

In [49]:
def process_geometry(file): 
    #================================== Data analysis ================================#
        # Read in the vertex positions of the geometry
        # Read in the indicies of the triangle mesh 
        vertex_positions, indicies = igl.read_triangle_mesh("%s%s"%(data_dir, file))

        print("filename: ", file)
        print("vertex_pos shape: ", vertex_positions.shape) 
        #print("vertex_pos head: \n", vertex_positions[:10])
        print("indicies shape: ", indicies.shape)
        #print("indicies head: \n", indicies[:10])
        #==================================#

        #================================== Display Cut ================================#
        boundary = igl.boundary_loop(indicies)
        #print("boundary length: ", len(boundary))

        rand = random.randint(0, len(indicies))
        seed_triangle_1 = indicies[rand] 
        seed_removed_indicies = np.delete(indicies, rand, axis=0)
        seed_triangle_2 = None
        if len(boundary) == 0:
            rand = random.randint(0, len(indicies))
            seed_triangle_2 = indicies[rand] 
            seed_removed_indicies = np.delete(seed_removed_indicies, random.randint(0, len(seed_removed_indicies)), axis=0)

        cut = igl.cut_to_disk(seed_removed_indicies)

        for sub_cut in cut:
            cut_vertices = vertex_positions[sub_cut]
            print("cut head: \n", sub_cut[:1], sub_cut[-1:], "\nlen: ", len(sub_cut))
            #print("cut points: \n", cut_vertices[:1], cut_vertices[-1:])
            #print("cut lines head: \n", np.array(tuple(zi(cut_vertices[:-1][:1], cut_vertices[1:][:1]))), np.array(tuple(zip(cut_vertices[:-1][:1], cut_vertices[1:][-1:])))

        colors = display_cut(vertex_positions=vertex_positions, indicies=indicies, full_cut=cut)
        #==================================#
        
        #================================== Display simple 2D parameterization ================================#
        #bnd = igl.boundary_loop(indicies)
        ##bnd = combine_cut(cut)
        ##print(bnd)
        ##colors = display_cut(vertex_positions=vertex_positions, indicies=indicies, full_cut=[bnd])
        ##bnd_uv = igl.map_vertices_to_circle(vertex_positions, bnd)
        ##print(len(bnd))
        ##print((bnd_uv))
        ##bnd_uv = np.array([circle_to_square(p[0],p[1]) for p in bnd_uv ])
        ##print(bnd_uv)

        #bnd_uv = []
        #bnd_uv.extend(zip(repeat(1.0), np.linspace(0, 1, int(len(bnd)/8), endpoint=False)))
        #bnd_uv.extend(zip(np.linspace(1, -1, int(len(bnd)/4 + len(bnd)%4), endpoint=False), repeat(1.0)))
        #bnd_uv.extend(zip(repeat(-1.0), np.linspace(1, -1, int(len(bnd)/4), endpoint=False)))
        #bnd_uv.extend(zip(np.linspace(-1, 1, len(bnd)-len(bnd_uv)-int(len(bnd)/8), endpoint=False), repeat(-1.0)))
        #bnd_uv.extend(zip(repeat(1.0), np.linspace(-1, 0, int(len(bnd)/8), endpoint=False)))
        #bnd_uv = np.array(bnd_uv)

        ## Create a harmonic parameterization for the inner vertices of the parameterized representation
        #print("running harmonic")
        #uv = igl.harmonic(vertex_positions, indicies, bnd, bnd_uv, 1)
        ##uv = igl.lscm(vertex_positions, indicies, bnd, bnd_uv)
        ##print(uv_h)
        #print(uv)
        #print("ran harmonic")
        #vertex_positions_p = np.hstack([uv, np.zeros((uv.shape[0],1))])

        #display_parameterized_geometry(vertex_positions=vertex_positions_p, indicies=indicies, colors=colors)
        #==================================#
        
        #================================== Trying combining the cut and doing cut_mesh ================================#
        #combined_cut = combine_cut(cut)
        #print("transforming cut")
        #cut = transform_cut(combined_cut, indicies)
        #vcut, fcut = igl.cut_mesh(vertex_positions, indicies, cut)
        #trimesh = tri.Trimesh(vertices=vcut, faces=fcut)
        #M = np.sum(np.mean(vertex_positions[indicies], axis=1), axis=1)
        #colors = tri.visual.interpolate(M, color_map='viridis')
        #trimesh.visual.face_colors = colors
        #scene = tri.Scene(trimesh)
        #scene.show(flags = {'cull': False}, viewer='gl')
        #bnd=cut 
        #bnd_uv = []
        #bnd_uv.extend(zip(repeat(1.0), np.linspace(0, 1, int(len(bnd)/8), endpoint=False)))
        #bnd_uv.extend(zip(np.linspace(1, -1, int(len(bnd)/4 + len(bnd)%4), endpoint=False), repeat(1.0)))
        #bnd_uv.extend(zip(repeat(-1.0), np.linspace(1, -1, int(len(bnd)/4), endpoint=False)))
        #bnd_uv.extend(zip(np.linspace(-1, 1, len(bnd)-len(bnd_uv)-int(len(bnd)/8), endpoint=False), repeat(-1.0)))
        #bnd_uv.extend(zip(repeat(1.0), np.linspace(-1, 0, int(len(bnd)/8), endpoint=False)))
        #bnd_uv = np.array(bnd_uv)

        #uv = igl.harmonic(vcut, fcut, cut, bnd_uv,1)
        #vcut = np.hstack([uv, np.zeros((uv.shape[0], 1))])
        #display_parameterized_geometry(vcut, fcut, colors=colors)
        #==================================#

        #================================== Try removing adjacent triangles ================================#
        indicies = remove_triangles_adjacent_to_path(indicies, cut)
        trimesh = tri.Trimesh(vertices=vertex_positions, faces=indicies)
        trimesh.remove_unreferenced_vertices()
        vertex_positions = trimesh.vertices
        indicies = trimesh.faces
        #M = np.sum(np.mean(vertex_positions[indicies], axis=1), axis=1)
        #colors = tri.visual.interpolate(M, color_map='viridis')
        #trimesh.visual.face_colors = colors
        #scene = tri.Scene(trimesh)
        #scene.show(flags = {'cull': False}, viewer='gl')

        bnd = igl.boundary_loop(indicies)
        #bnd = combine_cut(cut)
        #print(bnd)
        colors = display_cut(vertex_positions=vertex_positions, indicies=indicies, full_cut=[bnd])
        #bnd_uv = igl.map_vertices_to_circle(vertex_positions, bnd)
        #print(len(bnd))
        #print((bnd_uv))
        #bnd_uv = np.array([circle_to_square(p[0],p[1]) for p in bnd_uv ])
        #print(bnd_uv)

        bnd_uv = []
        bnd_uv.extend(zip(repeat(1.0), np.linspace(0, 1, int(len(bnd)/8), endpoint=False)))
        bnd_uv.extend(zip(np.linspace(1, -1, int(len(bnd)/4 + len(bnd)%4), endpoint=False), repeat(1.0)))
        bnd_uv.extend(zip(repeat(-1.0), np.linspace(1, -1, int(len(bnd)/4), endpoint=False)))
        bnd_uv.extend(zip(np.linspace(-1, 1, len(bnd)-len(bnd_uv)-int(len(bnd)/8), endpoint=False), repeat(-1.0)))
        bnd_uv.extend(zip(repeat(1.0), np.linspace(-1, 0, int(len(bnd)/8), endpoint=False)))
        bnd_uv = np.array(bnd_uv)

        # Create a harmonic parameterization for the inner vertices of the parameterized representation
        print("running harmonic")
        uv = igl.harmonic(vertex_positions, indicies, bnd, bnd_uv, 1)
        #uv = igl.lscm(vertex_positions, indicies, bnd, bnd_uv)
        #print(uv_h)
        print(uv)
        print("ran harmonic")
        vertex_positions_p = np.hstack([uv, np.zeros((uv.shape[0],1))])

        display_parameterized_geometry(vertex_positions=vertex_positions_p, indicies=indicies, colors=colors)
        #==================================#

        #================================== Display non-wireframe parameterization ================================#
        #trimesh = tri.Trimesh(vertices=vertex_positions_p, faces=indicies) 
        #trimesh.visual.face_colors = colors
        #scene = tri.Scene(trimesh)
        #scene.show(flags = {'cull': False}, viewer='gl')
        #==================================#


In [50]:
# print(dir(igl))

#"beetle.obj",
# "ChineseLion.obj",
#"Femur.obj",
#"Foot.obj",
#"happy.obj",
#"Hat.obj",
# "HumanBrain.obj",
#"HumanFace.obj",
#"Nefertiti.obj",
# "OldMan.obj",
#"rocker-arm.obj",
# "spot_triangulated.obj"
#"StanfordBunny.obj",
#"xyzrgb_dragon.obj"

process_geometry('Femur.obj')

filename:  Femur.obj
vertex_pos shape:  (21699, 3)
indicies shape:  (43301, 3)
cut head: 
 [15009] [15009] 
len:  4
cut head: 
 [8264] [8264] 
len:  96
cut head: 
 [15009] [8264] 
len:  273
running harmonic
[[-0.01045476  0.06276725]
 [ 0.07711092 -0.80489169]
 [-0.06391992 -0.86871605]
 ...
 [ 0.49546274 -0.50470762]
 [-0.40047929  0.0888452 ]
 [ 0.92718338 -0.30883962]]
ran harmonic


In [51]:
process_geometry('HumanBrain.obj')

filename:  HumanBrain.obj
vertex_pos shape:  (48463, 3)
indicies shape:  (96811, 3)
cut head: 
 [3788] [3788] 
len:  4
cut head: 
 [44453] [44453] 
len:  114
cut head: 
 [44453] [21269] 
len:  412
running harmonic
[[ 0.20055523  0.11994534]
 [ 0.23983067  0.10377693]
 [ 0.18593714  0.12491865]
 ...
 [-0.29145876 -0.24121212]
 [-0.31404084 -0.2450786 ]
 [-0.26894566 -0.25874425]]
ran harmonic


In [52]:
process_geometry('HumanFace.obj')

filename:  HumanFace.obj
vertex_pos shape:  (17157, 3)
indicies shape:  (34144, 3)
cut head: 
 [1602] [1602] 
len:  4
cut head: 
 [612] [612] 
len:  169
cut head: 
 [1602] [612] 
len:  175
running harmonic
[[-0.92075587 -0.77287971]
 [-0.75629585 -0.65173795]
 [-0.68716297 -0.59551983]
 ...
 [ 0.4473852   0.40713118]
 [ 0.44742052  0.40765553]
 [ 0.44752352  0.40692908]]
ran harmonic


In [53]:
process_geometry('spot_triangulated.obj')

filename:  spot_triangulated.obj
vertex_pos shape:  (2930, 3)
indicies shape:  (5856, 3)
cut head: 
 [561] [561] 
len:  4
cut head: 
 [1871] [1871] 
len:  4
cut head: 
 [1871] [561] 
len:  88
running harmonic
[[-0.5627681   0.21215309]
 [-0.75363084  0.54422643]
 [-0.3427867   0.45547982]
 ...
 [-0.77993967  0.66278671]
 [-0.78047881  0.66222866]
 [-0.77953517  0.66355958]]
ran harmonic
